In [12]:
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
import time
import pandas as pd

In [13]:
def get_all_business_link_in_story(story_url):
    """ This is the function that takes a particular story link from the website (black business business) and then scrolls through all business under
    this story, collects the business links and returns it"""
    # I used chrome; you can use whichever browser you like.
    browser = webdriver.Chrome()
    # Tell Selenium to get the URL you're interested in.
    browser.get(story_url)

    # Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

    # Now that the page is fully scrolled, grab the source code.
    source_data = browser.page_source

    # Throw your source into BeautifulSoup and start parsing!
    bsObj = BeautifulSoup(source_data)

    # Find all business url links in this particular story url
    url_tags = bsObj.find_all('a', {'class': 'u-url'})
    # compile a list of these url
    business_url_links = []
    for tag in url_tags:
        href = tag['href']
        business_url_links.append(href)
    return business_url_links

def getTitle(url_link):
    try:
        html = urlopen(url_link)
        bsObj = BeautifulSoup(html.read(), features="html.parser")
        title = bsObj.title
    except HTTPError as e:
        return None    
    except AttributeError as e:
        return None
    return title.text

def getBusinessInfo(url):
    try:
        html = urlopen(url)
        bsObj = BeautifulSoup(html.read(), features="html.parser")
        business_info = bsObj.find_all('em')
        business_info_text = [info.text for info in business_info]
    except HTTPError:
        return None
    except AttributeError:
        return None
    return business_info_text

story_url = "https://www.blackownedbrooklyn.com/stories?category=History%20and%20Culture"
story_business_url_links = get_all_business_link_in_story(story_url)
black_business_info = []
for url in story_business_url_links:
    url = "https://www.blackownedbrooklyn.com" + url
    title = getTitle(url_link=url)
    business_info = getBusinessInfo(url=url)
    black_business = {"name of business": title, "business contact and address": business_info}
    black_business_info.append(black_business)
    print(title)
    print(business_info)
    print("----------")
    


Kokomo — Black-Owned Brooklyn
['“', '65 Kent Ave, Brooklyn, ', '347- 799-1312', ', ', 'kokomonyc.com']
----------
Legacy — Black-Owned Brooklyn
['“We don’t preserve our culture as well as other people preserve it. So, for me, this store is about preserving culture', '247 Water St  Suite 104, Brooklyn, New York, 718- 210-2144, ', 'legacy dumbo.com']
----------
Juneteenth 2021 — Black-Owned Brooklyn
[]
----------
Hattie Carthan — Black-Owned Brooklyn
['more than 1,500', 'Magnolia Tree Earth Center, 677 Lafayette Ave, 718-387-2116, ', 'magnoliatreeearthcenter.com', 'Hattie Carthan Community Garden, 654A Lafayette Ave, 718-638-3566, ', 'hattiecarthancommunitymarket.com']
----------
Origins of AfroPunk  — Black-Owned Brooklyn
[]
----------
Juneteenth 2020 — Black-Owned Brooklyn
[]
----------
Tribute to the Ancestors of the Middle Passage — Black-Owned Brooklyn
[]
----------
bklyn boihood — Black-Owned Brooklyn
['bklynboihood.com']
----------
Here's to the Protesters (Black Lives Matter) — B

In [14]:
business_details_file = pd.DataFrame(black_business_info)
business_details_file

,name of business,business contact and address
0,Kokomo — Black-Owned Brooklyn,"[“, 65 Kent Ave, Brooklyn, , 347- 799-1312, , ..."
1,Legacy — Black-Owned Brooklyn,[“We don’t preserve our culture as well as oth...
2,Juneteenth 2021 — Black-Owned Brooklyn,[]
3,Hattie Carthan — Black-Owned Brooklyn,"[more than 1,500, Magnolia Tree Earth Center, ..."
4,Origins of AfroPunk — Black-Owned Brooklyn,[]
5,Juneteenth 2020 — Black-Owned Brooklyn,[]
6,Tribute to the Ancestors of the Middle Passage...,[]
7,bklyn boihood — Black-Owned Brooklyn,[bklynboihood.com]
8,Here's to the Protesters (Black Lives Matter) ...,[]
9,Remembering Lloyd Porter — Black-Owned Brooklyn,[]
